In [6]:
# 라이브러리
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
path = "C:/Users/gihun/To_raise_a_child_well/data/gihun/dataset/"

In [3]:
sum_df = pd.read_excel(path+'전국데이터프레임2006~2021.xlsx',index_col=0)
sum_df

,실업률,이혼율,고령화비율,혼인건수,소비자물가지수,출산아수/가임기여성의수,아파트전세지수,미분양주택지수,은행예금지수,경제활동참가율,지가변동률
시점,,,,,,,,,,,
1999-06-01,5.6,0.001352,NaN,36243.0,61.349,NaN,NaN,NaN,275077.7,49.0,NaN
1999-07-01,5.6,0.001913,NaN,25464.0,61.140,NaN,NaN,NaN,271995.7,48.7,NaN
1999-08-01,5.0,0.002503,NaN,19217.0,61.765,NaN,NaN,NaN,288399.4,48.1,NaN
1999-09-01,4.2,0.002985,NaN,16485.0,61.921,NaN,NaN,NaN,304701.1,49.2,NaN
1999-10-01,4.0,0.002003,NaN,24713.0,62.389,NaN,NaN,NaN,301666.2,49.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,2.5,NaN,17.8,NaN,108.930,0.001486,102.8,102.8,1958348.3,55.1,NaN
2022-10-01,2.4,NaN,17.9,NaN,109.210,0.001486,101.4,101.4,1967290.0,55.0,NaN
2022-11-01,2.3,NaN,17.9,NaN,109.100,0.001486,99.0,99.0,1970661.1,55.1,NaN


In [4]:
# 독립변수,종속변수 설정
X = sum_df.drop(columns='출산아수/가임기여성의수').loc['2006-01-01':'2021-12-01'].values
y = sum_df['출산아수/가임기여성의수'].loc['2006-01-01':'2021-12-01'].values
X.shape, y.shape

((192, 10), (192,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# 파라미터설정
param_grid = {'max_depth': [3, 4, 5],'learning_rate': [0.1, 0.01, 0.001],'n_estimators': [50, 100, 200]}

# 모델설정
cat = CatBoostRegressor()

# 그리드서치 파라미터 생성
grid = GridSearchCV(estimator=cat, param_grid=param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
# 출력
print("Best hyperparameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)

# 제일 좋은 것으로 다시돌리기
best_cat = CatBoostRegressor(max_depth=grid.best_params_['max_depth'],
                        learning_rate=grid.best_params_['learning_rate'],
                        n_estimators=grid.best_params_['n_estimators'])
best_cat.fit(X_train_scaled, y_train)

# 평가하기 
train_score = best_cat.score(X_train_scaled, y_train)
print("Train set score: ", train_score)
test_score = best_cat.score(X_test_scaled, y_test)
print("Test set score: ", test_score)

0:	learn: 0.0005235	total: 151ms	remaining: 7.41s
1:	learn: 0.0004928	total: 152ms	remaining: 3.64s
2:	learn: 0.0004624	total: 152ms	remaining: 2.38s
3:	learn: 0.0004375	total: 152ms	remaining: 1.75s
4:	learn: 0.0004117	total: 153ms	remaining: 1.37s
5:	learn: 0.0003890	total: 153ms	remaining: 1.12s
6:	learn: 0.0003682	total: 153ms	remaining: 942ms
7:	learn: 0.0003488	total: 154ms	remaining: 807ms
8:	learn: 0.0003316	total: 154ms	remaining: 701ms
9:	learn: 0.0003186	total: 154ms	remaining: 617ms
10:	learn: 0.0003042	total: 155ms	remaining: 548ms
11:	learn: 0.0002892	total: 155ms	remaining: 490ms
12:	learn: 0.0002757	total: 155ms	remaining: 441ms
13:	learn: 0.0002646	total: 155ms	remaining: 400ms
14:	learn: 0.0002537	total: 156ms	remaining: 363ms
15:	learn: 0.0002455	total: 156ms	remaining: 332ms
16:	learn: 0.0002374	total: 157ms	remaining: 304ms
17:	learn: 0.0002296	total: 157ms	remaining: 279ms
18:	learn: 0.0002236	total: 158ms	remaining: 257ms
19:	learn: 0.0002196	total: 158ms	remaini

In [8]:
print("Best hyperparameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)
print("Train set score: ", train_score)
print("Test set score: ", test_score)

Best hyperparameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Best score:  0.910217708119742
Train set score:  0.9958884537699494
Test set score:  0.9565062475984256
